## 3. Data Preprocessing

Prepare the data for annotation by performing basic preprocessing steps.

In [ ]:
# Basic preprocessing
print("Preprocessing data...")

# Prepare for annotation (simplified - no QC filtering)
dgsc.pp.prepare_for_annotation(adata, min_genes=200)

# Select highly variable genes
dgsc.pp.select_hvgs(adata, n_top_genes=500)  # Reduced for demo

print(f"After preprocessing: {adata.n_obs} cells × {adata.n_vars} genes")
print(f"Highly variable genes: {adata.var['highly_variable'].sum()}")

## 4. Clustering

Perform clustering to get initial cell groupings that will be used for annotation.

In [ ]:
# Principal component analysis
sc.tl.pca(adata, svd_solver='arpack')

# Compute the neighborhood graph
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

# Perform UMAP for visualization
sc.tl.umap(adata)

# Leiden clustering
sc.tl.leiden(adata, resolution=0.5)

# Also create a second clustering method for comparison
sc.tl.leiden(adata, resolution=0.3, key_added='leiden_03')

print(f"Leiden clusters (0.5): {adata.obs['leiden'].nunique()}")
print(f"Leiden clusters (0.3): {adata.obs['leiden_03'].nunique()}")

## 5. Visualize Initial Clustering

In [ ]:
# Plot UMAP with different annotations
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# True cell types
sc.pl.umap(adata, color='true_cell_type', ax=axes[0], show=False, title='True Cell Types')

# Leiden clustering
sc.pl.umap(adata, color='leiden', ax=axes[1], show=False, title='Leiden Clustering (0.5)')
sc.pl.umap(adata, color='leiden_03', ax=axes[2], show=False, title='Leiden Clustering (0.3)')

plt.tight_layout()
plt.show()

## 6. Create Marker Gene Sets

For demonstration, we'll create marker gene sets programmatically. In practice, you would load these from files.

In [ ]:
# Create marker gene sets for demonstration
import os

# Create markers directory if it doesn't exist
marker_dir = 'demo_markers'
os.makedirs(marker_dir, exist_ok=True)

# Create CellMarker dataset
cellmarker_data = {
    'gene': ['CD3D', 'CD3E', 'CD3G', 'CD19', 'CD79A', 'MS4A1', 'CD14', 'CD68', 'LYZ', 
             'CD8A', 'CD8B', 'CD4', 'FCGR3A', 'NCAM1'],
    'Tcell': [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
    'Bcell': [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    'Monocyte': [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    'NK': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
}

cellmarker_df = pd.DataFrame(cellmarker_data)
cellmarker_df.to_csv(f'{marker_dir}/CellMarker.csv', index=False)

# Create PanglaoDB dataset (slightly different markers)
panglaodb_data = {
    'gene': ['CD3D', 'CD3E', 'CD19', 'MS4A1', 'CD14', 'LYZ', 'CD8A', 'CD4', 'NCAM1'],
    'T_cell': [1, 1, 0, 0, 0, 0, 1, 1, 0],
    'B_cell': [0, 0, 1, 1, 0, 0, 0, 0, 0],
    'Myeloid': [0, 0, 0, 0, 1, 1, 0, 0, 0],
    'NK_cell': [0, 0, 0, 0, 0, 0, 0, 0, 1]
}

panglaodb_df = pd.DataFrame(panglaodb_data)
panglaodb_df.to_csv(f'{marker_dir}/PanglaoDB.tsv', sep='\t', index=False)

print(f"Created marker files in {marker_dir}/")
print(f"CellMarker.csv: {cellmarker_df.shape}")
print(f"PanglaoDB.tsv: {panglaodb_df.shape}")

## 7. Load Marker Gene Sets

In [ ]:
# Load marker gene sets
print("Loading marker gene sets...")
marker_sources = dgsc.load_markers(marker_dir)

print(f"Loaded {len(marker_sources)} marker sources:")
for source_name, markers in marker_sources.items():
    print(f"  {source_name}: {len(markers)} cell types")
    for cell_type, genes in markers.items():
        print(f"    {cell_type}: {len(genes)} genes")

## 8. Calculate Density Scores

Calculate density scores for different clustering methods and thresholds.

In [ ]:
# Calculate density scores
print("Calculating density scores...")

# Use multiple clustering methods and thresholds
dgsc.density_score(
    adata,
    marker_sources,
    cluster_columns=['leiden', 'leiden_03'],
    cutoffs=['0.5', 'mean', 'none']
)

# Check the new columns added
density_cols = [col for col in adata.obs.columns if any(source in col for source in marker_sources.keys())]
print(f"\nAdded {len(density_cols)} density score columns:")
for col in sorted(density_cols)[:10]:  # Show first 10
    print(f"  {col}")
if len(density_cols) > 10:
    print(f"  ... and {len(density_cols) - 10} more")

## 9. Visualize Density Scores

In [ ]:
# Plot density scores for one clustering method
score_cols = [col for col in adata.obs.columns if 'leiden_CellMarker' in col and '0.5' in col]

if len(score_cols) > 0:
    print(f"Plotting density scores for {len(score_cols)} cell types...")
    dgsc.pl.plot_density_scores(adata, score_cols[:4], 'leiden')  # Plot first 4
else:
    print("No density score columns found for visualization")

## 10. Deep Learning Annotation

Use deep learning to refine the annotations based on density scores.

In [ ]:
# Find a good annotation column for deep learning
annotation_candidates = [col for col in adata.obs.columns if 'Tcell' in col and '0.5' in col]

if len(annotation_candidates) > 0:
    annotation_col = annotation_candidates[0]
    print(f"Using annotation column: {annotation_col}")
    
    # Apply deep learning annotation
    print("Applying deep learning annotation...")
    dgsc.dgscrna_annotate(
        adata,
        annotation_column=annotation_col,
        epochs=5,  # Reduced for demo
        confidence_threshold=0.8
    )
    
    dl_annotation_col = f"{annotation_col}_DGscRNA"
    print(f"Deep learning annotation complete. Results in: {dl_annotation_col}")
    
    # Show annotation distribution
    print("\nAnnotation distribution:")
    print(adata.obs[dl_annotation_col].value_counts())
    
else:
    print("No suitable annotation column found for deep learning")
    dl_annotation_col = None

## 11. Run Complete Workflow

Alternatively, you can run the entire workflow with a single function call.

In [ ]:
# Create a copy of the data for the complete workflow
adata_workflow = adata.copy()

# Run the complete workflow
print("Running complete workflow...")
dgsc.run_dgscrna_workflow(
    adata_workflow,
    marker_folder=marker_dir,
    cluster_columns=['leiden'],
    cutoffs=['0.5'],
    epochs=3  # Reduced for demo
)

# Check results
workflow_cols = [col for col in adata_workflow.obs.columns if 'DGscRNA' in col]
print(f"\nWorkflow complete. Added {len(workflow_cols)} annotation columns:")
for col in workflow_cols:
    print(f"  {col}")

## 12. Compare Annotations

Compare the different annotation methods with the true cell types.

In [ ]:
# Compare annotations
if dl_annotation_col and dl_annotation_col in adata.obs.columns:
    print("Comparing annotations...")
    
    # Plot comparison
    dgsc.pl.plot_annotation_comparison(
        adata,
        'true_cell_type',
        dl_annotation_col
    )
    
    # Calculate accuracy
    from sklearn.metrics import accuracy_score, classification_report
    
    # Map annotations for comparison
    annotation_map = {
        'T_cell': 'Tcell',
        'B_cell': 'Bcell', 
        'Monocyte': 'Monocyte',
        'NK_cell': 'NK'
    }
    
    true_mapped = adata.obs['true_cell_type'].map(annotation_map)
    predicted = adata.obs[dl_annotation_col]
    
    # Only compare cells that have predictions
    mask = ~predicted.isin(['undecided', 'uncertain'])
    if mask.sum() > 0:
        accuracy = accuracy_score(true_mapped[mask], predicted[mask])
        print(f"\nAccuracy: {accuracy:.3f}")
        
        print("\nClassification Report:")
        print(classification_report(true_mapped[mask], predicted[mask]))

## 13. Visualization Gallery

Create comprehensive visualizations of the annotation results.

In [ ]:
# UMAP plots with different annotations
annotation_columns = ['true_cell_type', 'leiden']
if dl_annotation_col and dl_annotation_col in adata.obs.columns:
    annotation_columns.append(dl_annotation_col)

print("Creating UMAP visualization...")
dgsc.pl.plot_umap_annotations(adata, annotation_columns)

In [ ]:
# Plot marker gene expression
marker_genes = ['CD3D', 'CD19', 'CD14', 'FCGR3A']
available_markers = [gene for gene in marker_genes if gene in adata.var_names]

if len(available_markers) > 0:
    print(f"Plotting expression of {len(available_markers)} marker genes...")
    dgsc.pl.plot_marker_expression(adata, available_markers)

## 14. Summary and Results

Summarize the annotation results and provide insights.

In [ ]:
# Summary statistics
print("=== ANNOTATION SUMMARY ===")
print(f"Total cells: {adata.n_obs}")
print(f"Total genes: {adata.n_vars}")

print("\nTrue cell type distribution:")
print(adata.obs['true_cell_type'].value_counts())

print("\nLeiden clustering distribution:")
print(adata.obs['leiden'].value_counts())

if dl_annotation_col and dl_annotation_col in adata.obs.columns:
    print(f"\nDeep learning annotation distribution ({dl_annotation_col}):")
    print(adata.obs[dl_annotation_col].value_counts())

# List all annotation columns created
all_annotation_cols = [col for col in adata.obs.columns 
                      if any(source in col for source in marker_sources.keys()) or 'DGscRNA' in col]

print(f"\nCreated {len(all_annotation_cols)} annotation columns:")
for col in sorted(all_annotation_cols):
    unique_vals = adata.obs[col].nunique()
    print(f"  {col}: {unique_vals} unique values")

## 15. Save Results

Save the annotated data for further analysis.

In [ ]:
# Save the annotated data
output_file = 'annotated_data.h5ad'
adata.write(output_file)
print(f"Saved annotated data to: {output_file}")

# Save workflow results as well
workflow_output_file = 'annotated_data_workflow.h5ad'
adata_workflow.write(workflow_output_file)
print(f"Saved workflow results to: {workflow_output_file}")

# Clean up demo files
import shutil
shutil.rmtree(marker_dir)
print(f"Cleaned up demo marker directory: {marker_dir}")

## 16. Next Steps

This notebook demonstrated the complete `scanpy-dgscrna` workflow. Here are some next steps you can take:

### For Your Own Data:
1. **Load your h5ad file**: Replace the synthetic data creation with `sc.read_h5ad('your_data.h5ad')`
2. **Prepare marker files**: Organize your marker gene sets in the required format
3. **Adjust parameters**: Tune clustering resolution, epochs, confidence thresholds
4. **Evaluate results**: Compare with known cell types or manual annotations

### Advanced Usage:
- **Custom thresholds**: Experiment with different density score cutoffs
- **Multiple clustering methods**: Use different clustering algorithms
- **Marker integration**: Combine multiple marker databases
- **Validation**: Use cross-validation or hold-out sets

### Troubleshooting:
- **Memory issues**: Reduce the number of genes or use batch processing
- **Poor annotations**: Check marker gene quality and expression levels
- **Training issues**: Adjust learning rate, epochs, or network architecture

For more information, see the [documentation](https://github.com/yourusername/scanpy-dgscrna) or create an issue on GitHub.